In [3]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

from numpy import genfromtxt


import pandas as pd

from mpl_toolkits import mplot3d
#%matplotlib notebook

import math

#import cv2
import pims
from PIL import Image

import skimage

import json

#own modules / functions
import sys

sys.path.append("./modules")
import TiffProcessing as tiffproc
import InputProcessing as inproc
sys.path.append("./classes")
import Track

# Functions

In [4]:
inputParameters = pd.read_csv('../_inputParameters.csv',names=['key','value'])


inputDataFolder = inproc.getInputParameter(inputParameters,"inputDataFolder")
outputDataFolder = inproc.getInputParameter(inputParameters,"outputDataFolder")

trackingCsvFilename = inproc.getInputParameter(inputParameters,"trackingCsvFilename")
trackingCsvFilenameProcessed = inproc.getInputParameter(inputParameters,"trackingCsvFilenameProcessed")

slave_outputDataFolder  = inproc.getInputParameter(inputParameters,"slave_outputDataFolder")
master_outputDataFolder  = inproc.getInputParameter(inputParameters,"master_outputDataFolder")

slave_uniqueFilenameString = inproc.getInputParameter(inputParameters,"slave_uniqueFilenameString")
master_uniqueFilenameString = inproc.getInputParameter(inputParameters,"master_uniqueFilenameString")


slave_trackingProcessedBildFilename = inproc.getInputParameter(inputParameters,"slave_trackingProcessedBildFilename")
master_trackingProcessedBildFilename = inproc.getInputParameter(inputParameters,"master_trackingProcessedBildFilename")



framerate_msec = float(inproc.getInputParameter(inputParameters,"framerate_msec"))
movieLength = int(inproc.getInputParameter(inputParameters,"movieLength"))






In [42]:
allTrackData[0:200]

,trackId,tracklength,time[s],frameId,lifetime,catIdx,m_x,m_y,m_z,m_A,m_c,m_pval_Ar,s_x,s_y,s_z,s_A,s_c,s_pval_Ar
0,1,83,0,1,398.23,8,7.904661e+01,6.464731e+01,8.131298e+01,2.380362e+04,5.097926e+03,0,7.861783e+01,6.335770e+01,8.131298e+01,1.455107e+04,1.618403e+04,9.994436e-01
1,1,83,5.689000e+00,2,398.23,8,7.850739e+01,6.542015e+01,8.129176e+01,2.104784e+04,5.305183e+03,0,7.755419e+01,6.464906e+01,8.129176e+01,1.635423e+04,1.955088e+04,7.208670e-21
2,1,83,1.137800e+01,3,398.23,8,7.839408e+01,6.518394e+01,8.116666e+01,2.085433e+04,5.234626e+03,0,7.735373e+01,6.428964e+01,8.116666e+01,7.358672e+03,1.554692e+04,1
3,1,83,1.706700e+01,4,398.23,8,7.802180e+01,6.560145e+01,8.168228e+01,2.318226e+04,6.263921e+03,0,7.704685e+01,6.395467e+01,8.168228e+01,9.978225e+03,1.407360e+04,1
4,1,83,2.275600e+01,5,398.23,8,7.721728e+01,6.629176e+01,8.136034e+01,2.259943e+04,5.388422e+03,0,7.959930e+01,6.657997e+01,8.136034e+01,1.417622e+04,1.468696e+04,3.412941e-172
5,1,83,2.844500e+01,6,398.23,8,7.779343e+01,6.591954e+01,8.137399e+01,2.499614e+04,6.208032e+03,0,7.701031e+01,6.443481e+01,8.137399e+01,1.477947e+04,1.775916e+04,4.272127e-93
6,1,83,3.413400e+01,7,398.23,8,7.733113e+01,6.593743e+01,8.143128e+01,4.008776e+04,5.839431e+03,0,7.718457e+01,6.581676e+01,8.143128e+01,3.071578e+04,1.887844e+04,4.883844e-212
7,1,83,3.982300e+01,8,398.23,8,7.750617e+01,6.631077e+01,8.189304e+01,3.781167e+04,5.906465e+03,0,7.732744e+01,6.565193e+01,8.189304e+01,2.423569e+04,1.790760e+04,1.567365e-41
8,1,83,4.551200e+01,9,398.23,8,7.729240e+01,6.654524e+01,8.115095e+01,4.784633e+04,5.595024e+03,0,7.718690e+01,6.501774e+01,8.115095e+01,6.936350e+04,1.923998e+04,3.347619e-306
9,1,83,5.120100e+01,10,398.23,8,7.831880e+01,6.626546e+01,8.109910e+01,3.905074e+04,6.022057e+03,0,7.782275e+01,6.552805e+01,8.109910e+01,4.011041e+04,1.866853e+04,4.487719e-250


In [40]:
#%%time
# read the raw csv data for further processing

path = outputDataFolder+'/'+trackingCsvFilenameProcessed
print(path)
allTrackData = pd.read_csv(path,header=0)
allTrackData.columns = ["trackId","tracklength","time[s]","frameId","lifetime","catIdx","m_x",'m_y',"m_z","m_A","m_c","m_pval_Ar","s_x",'s_y',"s_z","s_A","s_c","s_pval_Ar"]


featureNames = ['trackId','trackLenght','lifetime','catIdx','meanFrameId', #early or late track
                'meanX','meanY','meanZ','varX','varY','varZ',
                'meanVelocity','varVelocity',
                'distanceTraveled',
                'm_meanIntensity','m_maxIntensity','m_minIntensity','m_varIntensity','m_intensityAUC','m_intensityDiffFirstLast',
                's_meanIntensity','s_maxIntensity','s_minIntensity','s_varIntensity','s_intensityAUC','s_intensityDiffFirstLast'] #intensity AUC will be high for triangle shaped


features = pd.DataFrame(columns=featureNames)

print(len(featureNames))

allTracks = np.sort(np.unique(allTrackData['trackId'].values))

for trackId in allTracks:
    trackData = allTrackData[allTrackData['trackId'] == trackId]
    trackLength = trackData['tracklength'].values[0]
    lifetime = trackData['lifetime'].values[0]
    catIdx = trackData['catIdx'].values[0]
    meanFrameId = np.nanmean(trackData['frameId'].values.astype(float))
    meanX = np.nanmean(trackData['m_x'].values.astype(float))
    meanY = np.nanmean(trackData['m_y'].values.astype(float))
    meanZ = np.nanmean(trackData['m_z'].values.astype(float))
    varX = np.nanvar(trackData['m_z'].values.astype(float))
    varY = np.nanvar(trackData['m_z'].values.astype(float))
    varZ = np.nanvar(trackData['m_z'].values.astype(float))
    start = np.array([trackData['m_x'].values.astype(float)[0],trackData['m_y'].values.astype(float)[0],trackData['m_z'].values.astype(float)[0]])
    end = np.array([trackData['m_x'].values.astype(float)[-1],trackData['m_y'].values.astype(float)[-1],trackData['m_z'].values.astype(float)[-1]])
    dist = np.linalg.norm(start-end)
    meanVelocity = dist/trackLength #px/frame
    varVelocity = 1
    distanceTraveled= dist
    m_meanIntensity = np.nanmean(trackData['m_A'].values.astype(float))
    m_maxIntensity = np.nanmax(trackData['m_A'].values.astype(float))
    m_minIntensity = np.nanmin(trackData['m_A'].values.astype(float))
    m_varIntensity = np.nanvar(trackData['m_A'].values.astype(float))
    m_intensityAUC = 1
    m_intensityDiffFirstLast = trackData['s_A'].values.astype(float)[0]-trackData['m_A'].values.astype(float)[-1]
    s_meanIntensity = np.nanmean(trackData['s_A'].values.astype(float))
    s_maxIntensity = np.nanmax(trackData['s_A'].values.astype(float))
    s_minIntensity = np.nanmin(trackData['s_A'].values.astype(float))
    s_varIntensity = np.nanvar(trackData['s_A'].values.astype(float))
    s_intensityAUC = 1
    s_intensityDiffFirstLast = trackData['s_A'].values.astype(float)[0]-trackData['s_A'].values.astype(float)[-1]
    
    
    
    #test  = [trackId,trackLength,catIdx,meanFrameId,meanX,meanY,meanZ,varX,varY,varZ,meanVelocity,varVelocity,distanceTraveled,
    #                         m_meanIntensity,m_maxIntensity,m_minIntensity,m_varIntensity,m_intensityAUC,m_intensityDiffFirstLast,
    #                        s_meanIntensity,s_maxIntensity,s_minIntensity,s_varIntensity,s_intensityAUC,s_intensityDiffFirstLast]
    #print(len(test))
    
    features.loc[trackId] = [trackId,trackLength,lifetime,catIdx,meanFrameId,meanX,meanY,meanZ,varX,varY,varZ,meanVelocity,varVelocity,distanceTraveled,
                             m_meanIntensity,m_maxIntensity,m_minIntensity,m_varIntensity,m_intensityAUC,m_intensityDiffFirstLast,
                            s_meanIntensity,s_maxIntensity,s_minIntensity,s_varIntensity,s_intensityAUC,s_intensityDiffFirstLast]
    
path = outputDataFolder+'/features.csv'
print(path)    
features.to_csv(path)  

/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100_bleachCorrected/output/trackedFeaturesProcessed.csv
26
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100_bleachCorrected/output/features.csv


In [41]:
features[1:10]

,trackId,trackLenght,lifetime,catIdx,meanFrameId,meanX,meanY,meanZ,varX,varY,...,m_minIntensity,m_varIntensity,m_intensityAUC,m_intensityDiffFirstLast,s_meanIntensity,s_maxIntensity,s_minIntensity,s_varIntensity,s_intensityAUC,s_intensityDiffFirstLast
2,2.0,70.0,398.23,4.0,35.500000,79.372957,56.960419,74.201671,4.067715,4.067715,...,3570.4720,1.960883e+07,1.0,883.2950,13313.682013,49200.53,659.4469,1.018641e+08,1.0,-3453.4480
3,3.0,75.0,398.23,8.0,33.878378,76.359349,53.252091,64.540903,6.129564,6.129564,...,1640.9140,1.305688e+07,1.0,-205.6310,9123.233941,26337.41,-4859.4180,5.139327e+07,1.0,778.8760
4,4.0,70.0,398.23,4.0,35.500000,77.811600,34.683369,57.680825,3.166868,3.166868,...,5913.4410,1.509706e+07,1.0,9659.2440,20370.355614,48645.96,7298.4650,9.843705e+07,1.0,7627.9620
5,5.0,77.0,398.23,8.0,37.394737,53.940486,81.285509,47.803007,12.429035,12.429035,...,1265.9440,6.670230e+06,1.0,15078.2520,22044.400103,64455.72,-473.7413,1.180987e+08,1.0,12002.8010
6,6.0,70.0,398.23,4.0,35.500000,61.437950,53.624291,61.548703,2.125682,2.125682,...,-2062.3170,2.112251e+06,1.0,3674.1570,3119.309795,12638.48,-5177.7690,1.921954e+07,1.0,2879.1080
7,7.0,77.0,398.23,8.0,37.947368,77.910633,45.208284,80.346601,2.317116,2.317116,...,921.4355,1.951189e+07,1.0,19442.6370,14302.650464,39904.40,-2096.4520,8.930447e+07,1.0,-4115.7000
8,8.0,175.0,398.23,8.0,26.175439,62.664925,72.392445,50.546598,31.572488,31.572488,...,-1844.8190,1.144191e+07,1.0,16868.0530,6252.512992,84495.73,-8216.2030,1.502214e+08,1.0,17499.9950
9,9.0,201.0,398.23,8.0,40.845361,40.730530,36.348650,37.518554,9.525391,9.525391,...,-1053.9970,1.610730e+07,1.0,7087.2110,9991.051873,47977.99,-7415.1070,1.358664e+08,1.0,-3248.0150
10,10.0,84.0,398.23,8.0,31.036145,75.015123,73.945040,47.106017,27.733442,27.733442,...,-583.8882,3.633224e+06,1.0,-4571.0844,5176.402708,26330.41,-6273.0160,4.567738e+07,1.0,-4426.2604
